In [2]:
import torch
import torch.nn as nn
import torchvision
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms, models
from sklearn.metrics import confusion_matrix, roc_auc_score
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

NameError: name '_C' is not defined

In [14]:
data_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # 調整解析度
    transforms.ToTensor(),
])
image_data = torchvision.datasets.ImageFolder('/Users/liushiwen/Desktop/大四上/機器學習導論/MachineLearning/hw4/reference', transform=data_transform)
train_loader = DataLoader(image_data, batch_size=4, shuffle=True)
print(len(train_loader))

25


In [3]:
print(image_data.class_to_idx)
print(image_data)

{'cats': 0, 'dogs': 1}
Dataset ImageFolder
    Number of datapoints: 100
    Root location: /Users/liushiwen/Desktop/大四上/機器學習導論/MachineLearning/hw4/reference
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=warn)
               ToTensor()
           )


In [4]:
print(image_data[0][0])
# display(image_data[0][0])

tensor([[[0.1529, 0.1569, 0.1608,  ..., 0.8078, 0.8000, 0.7843],
         [0.1569, 0.1569, 0.1608,  ..., 0.7922, 0.7843, 0.7765],
         [0.1529, 0.1529, 0.1490,  ..., 0.7725, 0.7804, 0.7882],
         ...,
         [0.1137, 0.1059, 0.0902,  ..., 0.1686, 0.1569, 0.1922],
         [0.1216, 0.1098, 0.0902,  ..., 0.1529, 0.1647, 0.2039],
         [0.1216, 0.1059, 0.0863,  ..., 0.2078, 0.1843, 0.1725]],

        [[0.1725, 0.1725, 0.1765,  ..., 0.8000, 0.7882, 0.7765],
         [0.1765, 0.1725, 0.1765,  ..., 0.7765, 0.7725, 0.7647],
         [0.1725, 0.1686, 0.1647,  ..., 0.7569, 0.7647, 0.7725],
         ...,
         [0.1059, 0.0980, 0.0824,  ..., 0.1137, 0.1059, 0.1490],
         [0.1137, 0.1020, 0.0824,  ..., 0.1020, 0.1137, 0.1608],
         [0.1137, 0.0980, 0.0784,  ..., 0.1569, 0.1333, 0.1294]],

        [[0.1569, 0.1647, 0.1765,  ..., 0.6902, 0.6549, 0.6314],
         [0.1608, 0.1647, 0.1765,  ..., 0.6706, 0.6471, 0.6235],
         [0.1569, 0.1608, 0.1686,  ..., 0.6510, 0.6431, 0.

In [53]:

torch.manual_seed(42)
# Define the data transformations
mean = np.array([0.5, 0.5, 0.5])
std = np.array([0.25, 0.25, 0.25])
data_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    # transforms.Normalize(mean, std),
])

# Load the dataset
image_data = torchvision.datasets.ImageFolder('/Users/liushiwen/Desktop/大四上/機器學習導論/MachineLearning/hw4/reference', transform=data_transform)

# Split the dataset into training and validation sets
dataset_size = len(image_data)
train_size = int(0.8 * dataset_size)
val_size = dataset_size - train_size
train_dataset, val_dataset = random_split(image_data, [train_size, val_size])

# Create data loaders for training and validation sets
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)


In [23]:
print(len(train_loader))
print(len(val_loader))

20
5


In [49]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(32 * 56 * 56, 64)  # Adjust the input size based on your image size
        self.fc2 = nn.Linear(64, 2)  # Output size: 2 (for dog and cat)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.pool(x)
        x = x.view(-1, 32 * 56 * 56)  # Adjust the input size based on your image size
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x
model = SimpleCNN()


In [70]:
# Instantiate the model, loss function, and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
num_epochs = 30
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")
    
    # Validation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Validation Accuracy: {100 * correct / total}%")


Epoch 1/20, Loss: 0.6966755837202072
Validation Accuracy: 45.0%
Epoch 2/20, Loss: 0.6959164589643478
Validation Accuracy: 45.0%
Epoch 3/20, Loss: 0.696355578303337
Validation Accuracy: 45.0%
Epoch 4/20, Loss: 0.6964239805936814
Validation Accuracy: 45.0%
Epoch 5/20, Loss: 0.6994983226060867
Validation Accuracy: 55.0%
Epoch 6/20, Loss: 0.7002792119979858
Validation Accuracy: 45.0%
Epoch 7/20, Loss: 0.6930624932050705
Validation Accuracy: 45.0%
Epoch 8/20, Loss: 0.6979827374219895
Validation Accuracy: 55.0%
Epoch 9/20, Loss: 0.6940547287464142
Validation Accuracy: 45.0%
Epoch 10/20, Loss: 0.6943883031606675
Validation Accuracy: 45.0%
Epoch 11/20, Loss: 0.6954115778207779
Validation Accuracy: 45.0%
Epoch 12/20, Loss: 0.6939002215862274
Validation Accuracy: 45.0%
Epoch 13/20, Loss: 0.6947601526975632
Validation Accuracy: 55.0%
Epoch 14/20, Loss: 0.6973469853401184
Validation Accuracy: 45.0%
Epoch 15/20, Loss: 0.693847045302391
Validation Accuracy: 45.0%
Epoch 16/20, Loss: 0.697328409552574

In [78]:

model.eval()
all_labels = []
all_predictions = []
with torch.no_grad():
    for inputs, labels in train_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        all_labels.extend(labels.numpy())
        all_predictions.extend(predicted.numpy())
print(all_labels)
print(all_predictions)
# Confusion Matrix
conf_matrix = confusion_matrix(all_labels, all_predictions)
print("Confusion Matrix (Training Set):")
print(conf_matrix)

# AUROC
# Assuming that outputs[:, 1] corresponds to the probability of the positive class (e.g., cat)
auroc = roc_auc_score(all_labels, all_predictions)
print(f"AUROC (Training Set): {auroc}")

[0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Confusion Matrix (Training Set):
[[ 0 39]
 [ 0 41]]
AUROC (Training Set): 0.5


In [79]:
file_path = "/Users/liushiwen/Desktop/大四上/機器學習導論/MachineLearning/hw4/HW4.pth"
torch.save(model.state_dict(), file_path)

In [80]:
def test_model(model, image_folder):
    model.eval()
    predictions = []

    with torch.no_grad():
        for image_path in image_folder:
            img = Image.open(image_path)
            img = transforms.Resize((224, 224))(img)  # 
            img_tensor = transforms.ToTensor()(img)
            output = model(img_tensor)
            _, predicted = torch.max(output.data, 1)
            predictions.append(predicted.item())

    return predictions
test_path = "/Users/liushiwen/Desktop/大四上/機器學習導論/MachineLearning/hw4/test"
test_folder =[]
for ii in range(20):
    test_folder.append(f"{test_path}/pic{ii+1}.jpg")
# test_folder = [f'{test_path}/pic1.jpg', f'{test_path}/pic2.jpg', f'{test_path}/pic3.jpg']  # Replace with actual image paths
testing_model = SimpleCNN()
testing_model.load_state_dict(torch.load(file_path))
predictions = test_model(testing_model, test_folder)

# Display the predictions
for i, prediction in enumerate(predictions, start=1):
    print(f"Image {i}: Predicted class {prediction}")

Image 1: Predicted class 1
Image 2: Predicted class 1
Image 3: Predicted class 1
Image 4: Predicted class 1
Image 5: Predicted class 1
Image 6: Predicted class 1
Image 7: Predicted class 1
Image 8: Predicted class 1
Image 9: Predicted class 1
Image 10: Predicted class 1
Image 11: Predicted class 1
Image 12: Predicted class 1
Image 13: Predicted class 1
Image 14: Predicted class 1
Image 15: Predicted class 1
Image 16: Predicted class 1
Image 17: Predicted class 1
Image 18: Predicted class 1
Image 19: Predicted class 1
Image 20: Predicted class 1
